# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from alpaca_trade_api.rest import REST, TimeFrame

import yfinance as yf
import datetime as dt 

## Load API Keys from Environment Variables

In [2]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [3]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp('2022-04-11', tz="America/New_York").isoformat()
past_date = pd.Timestamp('2022-05-11', tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_bars(
    ticker,
    TimeFrame.Day,
    limit=None,
    start='2022-04-05',
    end='2022-05-10',
    #after=None,
    #until=None,
).df

start = dt.datetime.today()-dt.timedelta(35)
end = dt.datetime.today()
ticker = 'AAPL'
df = yf.download(ticker, start, end)
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-10,155.520004,156.740005,152.929993,154.509995,154.509995,115366700
2022-05-11,153.500000,155.449997,145.809998,146.500000,146.500000,142689800
2022-05-12,142.770004,146.199997,138.800003,142.559998,142.559998,182602000
2022-05-13,144.589996,148.100006,143.110001,147.110001,147.110001,113787000
2022-05-16,145.550003,147.520004,144.179993,145.539993,145.539993,86541200


In [4]:
# Drop Outer Table Level

# Use the drop function to drop extra columns
df = df.drop(columns=["Open", "High", "Low", "Volume"], axis=1)

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()
df.to_excel('AAPLRaw.xlsx')

In [5]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change(3).shift(-3).dropna()

# Display sample data
df.to_excel('aapl.xlsx')
aapl_returns.tail()

,Close,Adj Close
2022-05-05,-0.014416,-0.012968
2022-05-06,-0.068540,-0.068540
2022-05-09,-0.062475,-0.062475
2022-05-10,-0.047893,-0.047893
2022-05-11,-0.006553,-0.006553


In [6]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [7]:
# Get first topic
aapl_headlines, dates = get_headlines("aapl")

Fetching news about 'aapl'
******************************


In [8]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************


In [9]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************


In [10]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [11]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [12]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)

In [13]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
    }
)
topic_sentiments

,aapl_avg,trade_avg,economy_avg


In [14]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)
topic_sentiments

,aapl_avg,trade_avg,economy_avg


In [15]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
topic_sentiments.to_excel("topicSent.xlsx")
topic_sentiments

,Close,Adj Close,aapl_avg,trade_avg,economy_avg


In [16]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\pandas\io\formats\style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\pandas\io\formats\style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,Close,Adj Close,aapl_avg,trade_avg,economy_avg
Close,nan,nan,nan,nan,nan
Adj Close,nan,nan,nan,nan,nan
aapl_avg,nan,nan,nan,nan,nan
trade_avg,nan,nan,nan,nan,nan
economy_avg,nan,nan,nan,nan,nan
